<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/Normalized_BachChorales_HandsOnChapter_15_fix1_MSE_fix2_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving chorale_229.csv to chorale_229.csv


In [0]:
uploaded_validation = files.upload()

Saving chorale_000.csv to chorale_000.csv


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np

X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()

dataX=X_train_000[:-1]
dataY=X_train_000[1:]
dataValidationX=X_valid_229[:-1]
dataValidationY=X_valid_229[1:]

print(dataX[0:10])
print(dataY[0:10])
print(type(dataX))

scaler = MinMaxScaler( feature_range=(0, 1) )
dataXScaler = scaler.fit_transform(dataX)
dataYScaler = scaler.fit_transform(dataY)
print(dataXScaler[0:10])
print(dataYScaler[0:10])

inversedX = scaler.inverse_transform(dataXScaler[0:10])
print('inversed:')
print(inversedX[0:10])
print(scaler.inverse_transform(dataXScaler)[2:30])

scalerValidation = MinMaxScaler( feature_range=(0, 1) )
dataValidationX = scalerValidation.fit_transform(dataValidationX)
dataValidationY = scalerValidation.fit_transform(dataValidationY)
print(dataValidationX[0:5])
print(dataValidationY[0:5])


print(scaler.inverse_transform(dataXScaler)[2:30])


   note0  note1  note2  note3
0     74     70     65     58
1     74     70     65     58
2     74     70     65     58
3     74     70     65     58
4     75     70     58     55
5     75     70     58     55
6     75     70     60     55
7     75     70     60     55
8     77     69     62     50
9     77     69     62     50
   note0  note1  note2  note3
0     72     67     60     48
1     72     67     60     48
2     72     67     60     48
3     72     67     60     48
4     72     67     64     48
5     72     67     64     48
6     72     67     64     50
7     72     67     64     50
8     72     67     64     52
9     72     67     64     52
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]]
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]]
<class 'numpy.ndarray'>
[[0.44444444 0

In [19]:
# reference: https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/
from keras.preprocessing.sequence import TimeseriesGenerator
# make length as a hyperparameter and find it's optimal value via GridSearchCV?
generator = TimeseriesGenerator(dataXScaler, dataYScaler, length=3, batch_size=32)
validation_generator = TimeseriesGenerator(dataValidationX, dataValidationY, length=3, batch_size=32)
print(generator[0])


(array([[[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235]],

       [[0.44444444, 0.77777778, 1.        , 0.88235294],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235]],

       [[0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.58333333, 0.70588235]],

       [[0.55555556, 0.77777778, 0.41666667, 0.70588235],
        [0.55555556, 0.77777778, 0.58333333, 0.70588235],
   

In [20]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense, Dropout, BatchNormalization, TimeDistributed
# the dataset knows the number of features, e.g. 2
n_features = 4

# choose a number of time steps
n_steps = None

model = Sequential()
model.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
model.add(LSTM(64, activation='tanh', input_shape=(None, n_features), return_sequences=True))
model.add(BatchNormalization())

model.add(LSTM(32 , activation = 'tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(n_features, activation='softmax'))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit_generator(generator, epochs=500, validation_data=validation_generator)

Epoch 1/500
6/6 [==============================] - 3s 421ms/step - loss: 0.1612 - val_loss: 0.1786
Epoch 2/500
6/6 [==============================] - 0s 18ms/step - loss: 0.1348 - val_loss: 0.1266
Epoch 3/500
6/6 [==============================] - 0s 17ms/step - loss: 0.1284 - val_loss: 0.1248
Epoch 4/500
6/6 [==============================] - 0s 18ms/step - loss: 0.1253 - val_loss: 0.1269
Epoch 5/500
6/6 [==============================] - 0s 17ms/step - loss: 0.1203 - val_loss: 0.1234
Epoch 6/500
6/6 [==============================] - 0s 17ms/step - loss: 0.1225 - val_loss: 0.1211
Epoch 7/500
6/6 [==============================] - 0s 18ms/step - loss: 0.1218 - val_loss: 0.1227
Epoch 8/500
6/6 [==============================] - 0s 16ms/step - loss: 0.1164 - val_loss: 0.1222
Epoch 9/500
6/6 [==============================] - 0s 18ms/step - loss: 0.1202 - val_loss: 0.1196
Epoch 10/500
6/6 [==============================] - 0s 16ms/step - loss: 0.1141 - val_loss: 0.1194
Epoch 11/500
6/6 [

In [21]:
# demonstrate prediction
print(scaler.inverse_transform(dataXScaler)[2:30])

x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]]
[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 6

**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [22]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[74. 70. 65. 58.]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[71.65973  69.55494  53.853416 43.2744  ]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [71.65972653 69.55493957 53.85341865 43.27439876]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[71.34236 68.22179 53.20636 47.30871]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [71.65972653 69.55493957 53.85341865 43.27439876]
 [71.34236649 68.22178406 53.20635723 47.30870888]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[70.95206 66.34743 53.17611 51.62926]]
expected:  [75. 70. 58. 55.]



Input: 
[[74.         70.         65.         58.        ]
 [71.65972653 69.55493957 53.85341865 43.27439876]
 [71.34236649 68.22178406 53.20635723 47.30870888]
 [70.95205525 66.34743011 53.17610913 51.62926197]]
--------

In [0]:
from keras.layers import Dense, SimpleRNN

# Using SimpleRNN 
print(n_features)
model_rnn = Sequential()
model_rnn.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
model_rnn.add(SimpleRNN(100, input_shape=[None, n_features], return_sequences=True ))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(n_features, activation='softmax'))

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy')
model_rnn.fit_generator(generator, epochs=500, validation_data=validation_generator)


4
Epoch 1/500
6/6 [==============================] - 2s 410ms/step - loss: 2.8488 - val_loss: 2.7542
Epoch 2/500
6/6 [==============================] - 0s 11ms/step - loss: 2.6689 - val_loss: 2.6543
Epoch 3/500
6/6 [==============================] - 0s 17ms/step - loss: 2.6477 - val_loss: 2.6706
Epoch 4/500
6/6 [==============================] - 0s 17ms/step - loss: 2.6340 - val_loss: 2.6723
Epoch 5/500
6/6 [==============================] - 0s 16ms/step - loss: 2.6356 - val_loss: 2.6434
Epoch 6/500
6/6 [==============================] - 0s 17ms/step - loss: 2.6209 - val_loss: 2.6438
Epoch 7/500
6/6 [==============================] - 0s 16ms/step - loss: 2.6028 - val_loss: 2.6420
Epoch 8/500
6/6 [==============================] - 0s 16ms/step - loss: 2.6022 - val_loss: 2.6250
Epoch 9/500
6/6 [==============================] - 0s 17ms/step - loss: 2.6090 - val_loss: 2.6255
Epoch 10/500
6/6 [==============================] - 0s 16ms/step - loss: 2.5977 - val_loss: 2.6263
Epoch 11/500
6/6

In [0]:
# demonstrate prediction
x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model_rnn.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[72.33333333 70.22222222 67.         58.23529412]
 [72.33333333 70.22222222 67.         58.23529412]
 [73.66666667 70.22222222 60.         55.58823529]
 [73.66666667 70.22222222 60.         55.58823529]
 [73.66666667 70.22222222 62.         55.58823529]
 [73.66666667 70.22222222 62.         55.58823529]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 70.22222222 64.         55.58823529]
 [76.33333333 70.22222222 64.         55.58823529]
 [76.33333333 69.33333333 64.         55.58823529]
 [76.33333333 69.33333333 64.         55.58823529]
 [73.66666667 67.55555556 65.         49.41176471]
 [73.66666667 67.55555556 65.         49.41176471]
 [73.66666667 69.33333333 65.         49.41176471]
 [73.66666667 69.33333333 65.         49.41176471]
 [72.33333333 70.22222222 67.         47.64705882]
 [72.33333333 70.22222222 67.  

In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[72.33333333 70.22222222 67.         58.23529412]]
---------------
1/1 [==============================] - 1s 1s/step
Predicted Output:  [[75.26765  65.903206 55.87507  45.003094]]
expected:  [72.33333333 70.22222222 67.         58.23529412]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [75.26764822 65.90320635 55.87506586 45.00309411]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[78.15341 64.03464 55.79461 45.00003]]
expected:  [72.33333333 70.22222222 67.         58.23529412]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [75.26764822 65.90320635 55.87506586 45.00309411]
 [78.15341401 64.03463399 55.79460961 45.0000323 ]]
---------------
1/1 [==============================] - 0s 2ms/step
Predicted Output:  [[75.54515  64.27181  58.038734 45.010487]]
expected:  [73.66666667 70.22222222 60.         55.58823529]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [75.26764822 65.90320635 55.87

In [0]:
# using GRU
from keras.models import Sequential
from keras.layers import GRU, Dense

# choose a number of time steps
n_steps = None
n_features = 4



model_gru = Sequential()
model_gru.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
model_gru.add(GRU(64, activation='tanh', input_shape=(None, n_features), return_sequences=True))
model_gru.add(BatchNormalization())

model_gru.add(GRU(32 , activation = 'tanh'))
model_gru.add(BatchNormalization())
model_gru.add(Dropout(0.2))
model_gru.add(Dense(n_features, activation='softmax'))
model_gru.compile(optimizer='adam', loss='categorical_crossentropy')

# fit model
model_gru.fit_generator(generator, epochs=500, validation_data=validation_generator)

Epoch 1/500
6/6 [==============================] - 4s 746ms/step - loss: 3.2365 - val_loss: 3.7881
Epoch 2/500
6/6 [==============================] - 0s 15ms/step - loss: 3.0617 - val_loss: 3.1095
Epoch 3/500
6/6 [==============================] - 0s 25ms/step - loss: 2.9522 - val_loss: 2.9572
Epoch 4/500
6/6 [==============================] - 0s 16ms/step - loss: 2.8794 - val_loss: 2.9560
Epoch 5/500
6/6 [==============================] - 0s 16ms/step - loss: 2.8688 - val_loss: 2.8725
Epoch 6/500
6/6 [==============================] - 0s 17ms/step - loss: 2.8729 - val_loss: 2.8114
Epoch 7/500
6/6 [==============================] - 0s 15ms/step - loss: 2.8483 - val_loss: 2.7877
Epoch 8/500
6/6 [==============================] - 0s 17ms/step - loss: 2.8308 - val_loss: 2.8471
Epoch 9/500
6/6 [==============================] - 0s 17ms/step - loss: 2.8720 - val_loss: 2.8296
Epoch 10/500
6/6 [==============================] - 0s 18ms/step - loss: 2.8530 - val_loss: 2.8576
Epoch 11/500
6/6 [

In [0]:
# demonstrate prediction
x_input = dataXScaler[2:30]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[2:30])
yhat = model_gru.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[30])

[[72.33333333 70.22222222 67.         58.23529412]
 [72.33333333 70.22222222 67.         58.23529412]
 [73.66666667 70.22222222 60.         55.58823529]
 [73.66666667 70.22222222 60.         55.58823529]
 [73.66666667 70.22222222 62.         55.58823529]
 [73.66666667 70.22222222 62.         55.58823529]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 69.33333333 64.         51.17647059]
 [76.33333333 70.22222222 64.         55.58823529]
 [76.33333333 70.22222222 64.         55.58823529]
 [76.33333333 69.33333333 64.         55.58823529]
 [76.33333333 69.33333333 64.         55.58823529]
 [73.66666667 67.55555556 65.         49.41176471]
 [73.66666667 67.55555556 65.         49.41176471]
 [73.66666667 69.33333333 65.         49.41176471]
 [73.66666667 69.33333333 65.         49.41176471]
 [72.33333333 70.22222222 67.         47.64705882]
 [72.33333333 70.22222222 67.  

In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model_gru.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[72.33333333 70.22222222 67.         58.23529412]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[69.76914  66.819084 57.545616 48.070786]]
expected:  [72.33333333 70.22222222 67.         58.23529412]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [69.76913869 66.81908059 57.54561144 48.07078555]]
---------------
1/1 [==============================] - 0s 3ms/step
Predicted Output:  [[70.597855 66.4717   57.273823 48.025963]]
expected:  [72.33333333 70.22222222 67.         58.23529412]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [69.76913869 66.81908059 57.54561144 48.07078555]
 [70.59785545 66.47170234 57.27382016 48.02596428]]
---------------
1/1 [==============================] - 0s 6ms/step
Predicted Output:  [[71.425766 65.70524  57.289185 48.40898 ]]
expected:  [73.66666667 70.22222222 60.         55.58823529]



Input: 
[[72.33333333 70.22222222 67.         58.23529412]
 [69.76913869 66.81908059 